In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
OPENAI_API_KEY = ""
ANTHROPIC_API_KEY = ""

In [ ]:
import os

# Set your API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# Initialize OpenAI Chat Model
llm_openai = ChatOpenAI(model="gpt-4o")
llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")


## 1. Constructing the basic langGraph flow

In [ ]:
from typing import Annotated, List

from typing_extensions import TypedDict
from langchain.schema import HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[List[AIMessage | HumanMessage], add_messages]

graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State) -> State:
    bot_response = llm.invoke(state["messages"])
    print(state["messages"]+[bot_response])
    print("\n")
    return {"messages": [bot_response]}


# The first argument is the unique node name
# The second argument is the function or object that will be called
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

### Show the visual graph node

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Run the chatbot using "graph.stream"

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

# value["messages"][-1].content
# it is used to access the content of the last message 
# in a list stored under the key "messages" in a dictionary named value

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## 2. Let's enhance it where we will keep the conversation state using "graph.invoke"

In [ ]:
from typing import TypedDict, List, Annotated
from langchain.schema import HumanMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance


class AgentState(TypedDict):
    messages: Annotated[List[AIMessage | HumanMessage], add_messages]
    user_input: str

In [ ]:
state = AgentState(messages=[], user_input="")
print(f"Initial state: {state}")

### Now let's add functions to modify the state

In [ ]:
# Configure Qdrant (Local)
qdrant_host = "localhost"
qdrant_port = 6333
collection_name = "transformer_docs"

client = QdrantClient(host=qdrant_host, port=qdrant_port)

In [ ]:
import openai

# 1. Function to take the user input and store it in AgentState
def add_user_message(state: AgentState) -> AgentState:
    new_query = HumanMessage(content=user_input)
    return {"messages":[new_query]} # Append new message}

# Helper function for retrieve relevant text
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list[float]:
    response = openai.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# 2. Function to retrieve relevant text
def retrieve_relevant_text(state: AgentState) -> AgentState:
    
    # Step 1: Get the latest user message
    query = state['messages'][-1].content

    # Step 2: Embed the query
    query_vector = get_embedding(query)

    results = client.search(
        collection_name= collection_name,
        query_vector=query_vector,
        limit=3,  # Get top 3 similar results
    )

    extracted_data = [(res.payload["filename"], res.payload["text"]) for res in results]
    
    
    prompt = f""" Please answer the query: {state['messages'][-1].content}
    
    based on the provided data {extracted_data}
    
    Please provide the response based on the provided data and your previous responses if the new query is the same, nothing else.
    
    """
    
    rewritten_query = HumanMessage(content=prompt)
    
    return {
        "messages": rewritten_query
    }

def summarize_prompt(state: AgentState) -> AgentState:
    answer = state['messages'][-1].content
    print(f"the answer from agent 1: {answer}\n")
    
    AI_prompt = f"""You are a data scientist and AI expert. Your responsibility is to respond to user's query on a mobile app, 
    hence your response should be short yet insightful.

    The other data scientist has provided a long answer as follows:
    {answer}
    for this query: {state['user_input']}.

    However if you see the answer is too long for users using mobile app and therefore this long answer might intimidate or confuse users, please provide a shorter yet insightful answer.
    If you think it's useful to provide some details, please add on a URL link that the user can read it themselves. """
    
    return {
        "messages" : AI_prompt
    }

    
# 3. Function to call LLM and get response
def generate_ai_response(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])  # Call LLM API
    return {"messages":[response]}  # Append AI response


# 4. Function to call LLM and get response
def short_ai_response(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])  # Call LLM API
    return {"messages":[response],  # Append AI response
           "user_input": ""}  # Set back the user input to blank

### Let's create the graph instance to put together all the states and the functionalities

In [ ]:
# 1. Initiate the graph instance from the main class StateGraph that hooks up the data stored in AgentState class
graph = StateGraph(AgentState)

# 2. Define nodes
graph.add_node("add_user_message", add_user_message)
graph.add_node("retrieve_relevant_text", retrieve_relevant_text)
graph.add_node("generate_ai_response", generate_ai_response)  # Agent 1
graph.add_node("summarize_prompt", summarize_prompt)
# graph.add_node("short_ai_response", generate_ai_response)  # Agent 2
graph.add_node("short_ai_response", short_ai_response)  # Agent 2 : correction, it should be "short_ai_response"


# 3. Define edges (flow of the graph)
graph.add_edge(START, "add_user_message")
graph.add_edge("add_user_message", "retrieve_relevant_text")
graph.add_edge("retrieve_relevant_text", "generate_ai_response")
graph.add_edge("generate_ai_response", "summarize_prompt")
graph.add_edge("summarize_prompt", "short_ai_response")
graph.add_edge("short_ai_response", END)

# 4. Convert the graph structure into an executable flow
workflow = graph.compile() 

In [ ]:
from IPython.display import Image, display

try:
    display(Image(workflow.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Let's test and run the workflow

In [ ]:

# Initialize chatbot state and must the same class AgenState!
state = AgentState(messages=[], user_input="")
print(f"\nInitial state: {state}\n")


# Simulate conversation
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
        
    state["user_input"] = user_input
    state = workflow.invoke(state)  # Run the workflow and update state
    bot_response = state["messages"][-1].content  # Get last AI response
    print(f"The answer from Agent 2: {bot_response}")
    
#     print(f"\nUpdated state: {state}") # Checkpoint to check the updated state

### 0. Offline part: Setting up vector database and the data processing for data extraction and vector embedding

In [ ]:
import openai

from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

In [ ]:
# Configure Qdrant (Local)
qdrant_host = "localhost"
qdrant_port = 6333
collection_name = "transformer_docs"

client = QdrantClient(host=qdrant_host, port=qdrant_port)

In [ ]:
def preprocess_dataset(docs_list):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=700,
        chunk_overlap=50,
        disallowed_special=()
    )
    doc_splits = text_splitter.split_documents(docs_list)
    return doc_splits

In [ ]:
transformers_doc = HuggingFaceDatasetLoader("m-ric/transformers_documentation_en","text")

In [ ]:
transformer_splits = preprocess_dataset(transformers_doc.load()[:5])

In [ ]:
from pprint import pprint

for doc in transformer_splits:
    pprint(doc)

In [ ]:
# Prepare OpenAI Embeddings
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list[float]:
    response = openai.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

In [ ]:
if collection_name not in [c.name for c in client.get_collections().collections]:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )
else:
    print("Collection already exists!")

In [ ]:
from qdrant_client.models import Batch
import uuid

ids = []
vectors = []
payloads = []

# 1. Assign docs into list before uploading into Qdrant vector db
for doc in transformer_splits:
    text = doc.page_content
    metadata = doc.metadata
    
    vector = get_embedding(text)
    
    ids.append(str(uuid.uuid4()))
    vectors.append(vector)
    payloads.append(metadata | {"text": text})

    
# 2. Upload vectors and payloads into Qdrant vector db
client.upsert(
    collection_name = collection_name,
    points=Batch(
        ids=ids,
        vectors=vectors,
        payloads=payloads   
    )
)

In [ ]:
from qdrant_client.models import Filter
from pprint import pprint

# Scan through the stored documents and vector embedded
scroll_result, next_page = client.scroll(
    collection_name=collection_name,
    limit=1,
    with_payload=True,
    with_vectors=True,
    offset=None
)

for point in scroll_result:
    print(f"ID : {point.id}")
    pprint(point.payload)
    pprint(point.vector)